In [56]:
import pprint
import pandas as pd
# We can, as usual, access the examples directly.
from q2_diversity_lib import examples
ofe = examples.observed_features_example

# But probably the better way to do this is to get all examples from the plugin.
from qiime2.plugins import diversity_lib
examples = diversity_lib.methods.observed_features.examples
print(examples)
pofe = examples['basic']

{'basic': <function observed_features_example at 0x7f8d220319d0>}


In [23]:
from qiime2.sdk.usage import ExecutionUsage
# Execute the examples and look at the result
ex_use = ExecutionUsage()
eres = ofe(ex_use)
print(ex_use.render())

print()

ex2 = ExecutionUsage()
peres = pofe(ex2)
print(ex_use.render())

{'feature_table': <ExecutionUsageVariable name='feature_table', var_type='artifact'>, 'obs_feat_vector': <ExecutionUsageVariable name='obs_feat_vector', var_type='artifact'>}

{'feature_table': <ExecutionUsageVariable name='feature_table', var_type='artifact'>, 'obs_feat_vector': <ExecutionUsageVariable name='obs_feat_vector', var_type='artifact'>}


In [34]:
from q2cli.core.usage import CLIUsage
c_use = CLIUsage()
cres = ofe(c_use)
print(c_use.render())

# observed_features takes a basic qualitative or quantitative FeatureTable
# and no parameters. It returns a vector of per-sample alpha diversity
# values.
qiime diversity-lib observed-features \
  --i-table feature-table.qza \
  --o-vector obs-feat-vector.qza
# Our result vector is <CLIUsageVariable name='obs_feat_vector',
# var_type='artifact'>


In [33]:
from qiime2.plugins import ArtifactAPIUsage
# TODO: This blows up if moved above the ExecutionUsage block, b/c no PluginManager. Why/do we care?
art_use = ArtifactAPIUsage()
ares = ofe(art_use)
print(art_use.render())

print("\n-----------------------------------------------------------------------------------------------\n")

# NOTE: If we re-use a UsageDriver it will keep accreting stuff, making for a tangled render.
# Best to fire up a new instance, I suspect.
art_use = ArtifactAPIUsage()
pares = pofe(art_use)
print(art_use.render())

import qiime2.plugins.diversity_lib.actions as diversity_lib_actions

# observed_features takes a basic qualitative or quantitative FeatureTable and no parameters. It returns a vector of per-sample alpha diversity values.
obs_feat_vector, = diversity_lib_actions.observed_features(
    table=feature_table,
)
# Our result vector is <ArtifactAPIUsageVariable name='obs_feat_vector', var_type='artifact'>

-----------------------------------------------------------------------------------------------

import qiime2.plugins.diversity_lib.actions as diversity_lib_actions

# observed_features takes a basic qualitative or quantitative FeatureTable and no parameters. It returns a vector of per-sample alpha diversity values.
obs_feat_vector, = diversity_lib_actions.observed_features(
    table=feature_table,
)
# Our result vector is <ArtifactAPIUsageVariable name='obs_feat_vector', var_type='artifact'>


In [5]:
from qiime2.sdk.usage import DiagnosticUsage
# TODO: This blows up if moved above the ExecutionUsage block, b/c no PluginManager. Why/do we care?
d_use = DiagnosticUsage()
dres = ofe(d_use)
print(d_use)
print()
print(d_use.render())


[DiagnosticUsage.DiagnosticUsageRecord(source='comment', variable='observed_features takes a basic qualitative or quantitative FeatureTable and no parameters. It returns a vector of per-sample alpha diversity values.'), DiagnosticUsage.DiagnosticUsageRecord(source='init_artifact', variable=<UsageVariable name='feature_table', var_type='artifact'>), DiagnosticUsage.DiagnosticUsageRecord(source='action', variable=UsageOutputs (name = value)
--------------------------------------------------------------------
vector = <UsageVariable name='obs_feat_vector', var_type='artifact'>), DiagnosticUsage.DiagnosticUsageRecord(source='comment', variable="Our result vector is <UsageVariable name='obs_feat_vector', var_type='artifact'>")]


In [ ]:
===================================================================================================

In [6]:
import numpy as np
import biom

from qiime2 import Artifact
from qiime2.plugins import diversity_lib

In [7]:
# Manually build our result so we can see why the assert is failing - check! nasty whitespace.
input = Artifact.import_data(
    'FeatureTable[Frequency]',
    biom.Table(np.array([[1, 0, .5, 999, 1],
                        [0, 1, 2, 0, 5],
                        [0, 0, 0, 1, 10]]),
               ['A', 'B', 'C'],
               ['S1', 'S2', 'S3', 'S4', 'S5']))

r, = diversity_lib.methods.observed_features(input)
print(r)
# r.save('./obs_feat.qza')

<artifact: SampleData[AlphaDiversity] uuid: ea4f4f79-1715-42c0-ad1c-b97e2b956ce9>


In [50]:
from qiime2.plugins import diversity_lib
pofe = diversity_lib.methods.observed_features.examples
print(pofe)
print()
d_use = DiagnosticUsage()
pofe['basic'](d_use)
steps = d_use.render()
step1, step2, step3, step4 = steps
for step in steps:
    pprint.pp(step)
    print()
    
# These values happen to be here for the DiagnosticUsage driver,
# but ExecutionDriver (e.g.) has different attributes
print(step2.source)
print(step2.variable)


{'basic': <function observed_features_example at 0x7f8d220319d0>}

DiagnosticUsage.DiagnosticUsageRecord(source='comment', variable='observed_features takes a basic qualitative or quantitative FeatureTable and no parameters. It returns a vector of per-sample alpha diversity values.')

DiagnosticUsage.DiagnosticUsageRecord(source='init_artifact', variable=<UsageVariable name='feature_table', var_type='artifact'>)

DiagnosticUsage.DiagnosticUsageRecord(source='action', variable=UsageOutputs (name = value)
--------------------------------------------------------------------
vector = <UsageVariable name='obs_feat_vector', var_type='artifact'>)

DiagnosticUsage.DiagnosticUsageRecord(source='comment', variable="Our result vector is <UsageVariable name='obs_feat_vector', var_type='artifact'>")

init_artifact
<UsageVariable name='feature_table', var_type='artifact'>


In [70]:
## We can (and should not) save ExecutionUsageVariables

ex_use = ExecutionUsage()
eres = ofe(ex_use)
res = ex_use.render()
print(res)

# This is verboten... but it works!
# we are definitely making real Results with the ExecutionDriver
res['obs_feat_vector'].save('feature_data_deleteme.qza')


{'feature_table': <ExecutionUsageVariable name='feature_table', var_type='artifact'>, 'obs_feat_vector': <ExecutionUsageVariable name='obs_feat_vector', var_type='artifact'>}


'feature_data_deleteme.qza'